In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Подготовительное

Загрузка данных из репозитория соревнования, можно не запускать

In [ ]:
!git clone https://github.com/IlyaGusev/purano

fatal: destination path 'purano' already exists and is not an empty directory.


In [ ]:
%cd purano

/content/purano


In [ ]:
!pip install -r requirements.txt

In [ ]:
!mkdir models/lang_detect
!mkdir models/cat_detect
!wget https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz -O models/lang_detect/lang_detect_v10.ftz
!wget https://www.dropbox.com/s/23x35wuet280eh6/ru_cat_v5.ftz -O models/cat_detect/ru_cat_v5.ftz

mkdir: cannot create directory ‘models/lang_detect’: File exists
mkdir: cannot create directory ‘models/cat_detect’: File exists
--2021-03-23 14:05:06--  https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz [following]
--2021-03-23 14:05:06--  https://www.dropbox.com/s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc453ba23cc54704d2690b01c18e.dl.dropboxusercontent.com/cd/0/inline/BLOOqptPrJgKhUqOUB1AuBa2thaa0jtXNZswiYBSeeXMZDq6KHMmZei10T4cn5uhZoOEchI5pEGDN4upGSNrZY_iBpYN8gseTEJS9ovSU1mwo_uS7oTHL7Wu-zXd93cS8f0yERmjD6w_mdAXZvqZwHrl/file# [following]
--2021-03-23 14:05:06--  https://uc453ba

In [ ]:
# May 25 (training)
!rm -rf data
!mkdir -p data
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz -O - | tar -xz && cd ../
# May 27 and 29 (test)
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0527.tar.gz -O - | tar -xz && cd ../
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0529.tar.gz -O - | tar -xz && cd ../

--2021-03-23 14:05:17--  https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145541247 (139M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 138.80M  24.8MB/s    in 6.5s    

2021-03-23 14:05:24 (21.5 MB/s) - written to stdout [145541247/145541247]

--2021-03-23 14:05:25--  https://data-static.usercontent.dev/DataClusteringDataset0527.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166313405 (159M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[=======

In [ ]:
%%writefile configs/cleaner.jsonnet
{
    "lang_detect_model_path": "models/lang_detect/lang_detect_v10.ftz",
    "cat_detect_model_path": "models/cat_detect/ru_cat_v5.ftz",
    "is_lower": true,
    "is_russian_only": true,
    "is_news_only": true
}

Overwriting configs/cleaner.jsonnet


In [ ]:
!rm -f 0525_parsed.db
!python3 -m purano.run_parse --inputs "data/20200525" --output-file 0525_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet
!rm -f 0527_parsed.db
!python3 -m purano.run_parse --inputs "data/20200527" --output-file 0527_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet
!rm -f 0529_parsed.db
!python3 -m purano.run_parse --inputs "data/20200529" --output-file 0529_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet

Parsed 10000 documents
Parsed 20000 documents
Parsed 30000 documents
Parsed 40000 documents
Parsed 50000 documents
Parsed 60000 documents
Parsed 70000 documents
Parsed 80000 documents
19380 will be saved
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19380 entries, 3243 to 14993
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   url            19380 non-null  object                   
 1   host           19380 non-null  object                   
 2   title          19380 non-null  object                   
 3   text           19380 non-null  object                   
 4   date           19380 non-null  datetime64[ns, tzlocal()]
 5   patched_title  19380 non-null  object                   
 6   patched_text   19380 non-null  object                   
 7   category       19380 non-null  object                   
dtypes: datetime64[ns, tzlocal()](1), object(7)
memory usage: 1.3

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from purano.models import Document

def parse_db(file_name):
    db_engine = "sqlite:///{}".format(file_name)
    engine = create_engine(db_engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    query = session.query(Document)
    docs = query.all()
    records = []
    for doc in docs:
        records.append({
            "title": doc.title,
            "text": doc.text,
            "url": doc.url,
            "host": doc.host,
            "timestamp": doc.date,
            "patched_title": doc.patched_title,
            "patched_text": doc.patched_text
        })
    return records

train_records = parse_db("0525_parsed.db")
test_0527_records = parse_db("0527_parsed.db")
test_0529_records = parse_db("0529_parsed.db")
print(len(train_records))
print(len(test_0527_records))
print(len(test_0529_records))
print(train_records[0])

19380
20080
19096
{'title': 'Коронавирус закрыл круглосуточную центральную аптеку Бреста', 'text': 'Сообщение о том, что Брестская центральная районная аптека № 2 (пр. Машерова, 76) временно прекращает обслуживание, появилось 20 мая. И с тех пор ничего не поменялось. Брестчане безуспешно дергают за ручку входной двери. Все закрыто. Генеральный директор Брестского РУП «Фармация» Елена Ломеко пояснила ситуацию журналисту «Вечерки». - Аптека не работает в связи с тем, что ее специалисты были установлены как контакты первого уровня по COVID-19. И для них сейчас введен режим самоизоляции. Аптека будет закрыта до получения очередных результатов исследований. На сегодняшний день трем специалистам больничные закрыли, они вышли на работу. Это работники отдела запасов. Они занимаются сборкой и отправкой товаров по заявкам. Такая работа необходима, учитывая, что центральная районная аптека № 2 обслуживает еще и подчиненные сельские аптеки и ФАПы. Специалисты рецептурно-производственного отдела, к

In [ ]:
import pickle

with open('/content/drive/MyDrive/rusum/train_records.pickle', 'wb') as f:
  pickle.dump(train_records, f)
with open('/content/drive/MyDrive/rusum/test_0527_records.pickle', 'wb') as f:
  pickle.dump(test_0527_records, f)
with open('/content/drive/MyDrive/rusum/test_0529_records.pickle', 'wb') as f:
  pickle.dump(test_0529_records, f)

In [ ]:
train_records[0]['title']

'Коронавирус закрыл круглосуточную центральную аптеку Бреста'

In [ ]:
train_records[0]['patched_title']

'коронавирус закрыл круглосуточную центральную аптеку бреста'


#Делаем датасет

In [ ]:
import pickle

with open('/content/drive/MyDrive/rusum/train_records.pickle', 'rb') as f:
  train_records = pickle.load(f)
with open('/content/drive/MyDrive/rusum/test_0527_records.pickle', 'rb') as f:
  test_0527_records = pickle.load(f)
with open('/content/drive/MyDrive/rusum/test_0529_records.pickle', 'rb') as f:
  test_0529_records = pickle.load(f)

In [ ]:
train_records[0]

NameError: ignored

In [ ]:
import pandas as pd

train = pd.read_csv('/content/drive/MyDrive/rusum/ru_clustering_0525_urls.tsv', sep='\t')
test = pd.read_csv('/content/drive/MyDrive/rusum/ru_clustering_test.tsv', sep='\t')

In [ ]:
test[:3]

In [ ]:
train_texts[3]

In [ ]:
train_labels[3]

In [ ]:
len(train_texts)

In [ ]:
train_texts = []
train_labels = []

def search(url, st):
  if st=='train':
    for tr in train_records:
      if tr['url']==url:
        return ' '.join((tr['title']+' '+tr['text']).split()[:200])
  else:
    for tr in test_0527_records:
      if tr['url']==url:
        return ' '.join((tr['title']+' '+tr['text']).split()[:200])
    for tr in test_0529_records:
      if tr['url']==url:
        return ' '.join((tr['title']+' '+tr['text']).split()[:200])


for line in train.values:
  train_texts.append([search(line[0],'train'),search(line[1],'train')])
  if line[2]=='OK':
    train_labels.append(1)
  else:
    train_labels.append(0)

In [ ]:
test_texts = []

for line in test.values:
  test_texts.append([search(line[1],'test'),search(line[2],'test')])

del(train_records)
del(test_0527_records)
del(test_0529_records)

In [ ]:
!pip install num2words

In [ ]:
import num2words
from num2words import num2words

def digit_part(text):
  isd = True
  for s in text:
    if not s.isdigit():
      isd = False
      break
  if isd:
    return num2words(text, lang = 'ru')
  else:
    return text

text = 'я люблю 42'
text = ' '.join([digit_part(word) for word in text.split()])
text

In [ ]:
test_texts[9]

In [ ]:
for i in range(len(train_texts)):
  train_texts[i][0] = ' '.join([digit_part(word) for word in train_texts[i][0].split()])
  train_texts[i][1] = ' '.join([digit_part(word) for word in train_texts[i][1].split()])

In [ ]:
for i in range(len(test_texts)):
  try:
    test_texts[i][0] = ' '.join([digit_part(word) for word in test_texts[i][0].split()])
  except:
    print(test_texts[i])
  try:
    test_texts[i][1] = ' '.join([digit_part(word) for word in test_texts[i][1].split()])
  except:
    print(test_texts[i])

#Модель

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
!pip install transformers
!pip install pytorch-pretrained-bert pytorch-nlp
from transformers import BertModel#, RobertaModel
import numpy as np
import tensorflow as tf

from transformers import *
import pandas as pd
#import torch
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import os
import pickle

In [ ]:
model_name = 'DeepPavlov/rubert-base-cased'
#model_name = "dmitry-vorobiev/rubert_ria_headlines"
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for i, sent in enumerate(train_texts):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    sent = '[CLS] '+ sent[0] + ' [SEP] ' +sent[1] + ' [SEP]'
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        truncation = True,
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', train_texts[0])
print('Token IDs:', input_ids[0])
del(train_texts)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  ['На мужчину, заразившего коронавирусом двух постояльцев хостела в Москве, завели дело На мужчину, недолечившего коронавирусную инфекцию и заразившего ею двух постояльцев хостела в Москве, завели уголовное дело по части один статьи двести тридцать шесть УК РФ («Нарушение санитарно-эпидемиологических правил, повлекшее по неосторожности массовое заболевание»). Об этом сообщается на сайте генпрокуратуры. «Установлено, что мужчина, заболевший коронавирусной инфекцией, выписан для продолжения лечения в амбулаторных условиях. При выписке он предоставил в больницу недостоверную информацию об адресе проживания и прибыл для проживания в хостел, скрыв от администрации информацию о своем заболевании, вследствие чего возникла угроза заражения COVID-19 других лиц. Кроме того, после выявления данного случая по результатам анализов заболевание диагностировано у двух постояльцев, контактировавших с мужчиной», — передает пресс-служба ведомства. Генеральному директору хостела прокурор объявил

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

13,354 training samples
1,484 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 4

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import RobertaForSequenceClassification, AdamW, RobertaConfig

model = BertForSequenceClassification.from_pretrained(
    model_name, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (119547, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              

In [ ]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)[0]
        logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)[1]                   

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            loss = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)[0]
            logits = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)[1]
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...
  Batch    40  of  3,339.    Elapsed: 0:00:28.
  Batch    80  of  3,339.    Elapsed: 0:00:55.
  Batch   120  of  3,339.    Elapsed: 0:01:21.
  Batch   160  of  3,339.    Elapsed: 0:01:48.
  Batch   200  of  3,339.    Elapsed: 0:02:15.
  Batch   240  of  3,339.    Elapsed: 0:02:42.
  Batch   280  of  3,339.    Elapsed: 0:03:09.
  Batch   320  of  3,339.    Elapsed: 0:03:36.
  Batch   360  of  3,339.    Elapsed: 0:04:02.
  Batch   400  of  3,339.    Elapsed: 0:04:29.
  Batch   440  of  3,339.    Elapsed: 0:04:56.
  Batch   480  of  3,339.    Elapsed: 0:05:23.
  Batch   520  of  3,339.    Elapsed: 0:05:49.
  Batch   560  of  3,339.    Elapsed: 0:06:16.
  Batch   600  of  3,339.    Elapsed: 0:06:43.
  Batch   640  of  3,339.    Elapsed: 0:07:10.
  Batch   680  of  3,339.    Elapsed: 0:07:36.
  Batch   720  of  3,339.    Elapsed: 0:08:03.
  Batch   760  of  3,339.    Elapsed: 0:08:30.
  Batch   800  of  3,339.    Elapsed: 0:08:57.
  Batch   840  of


#Тест

In [ ]:
import pandas as pd

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in test_texts:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    sent = '[CLS] '+ sent[0] + ' [SEP] ' +sent[1] #+ ' [SEP]'

    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        truncation = True,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(val_labels)

# Set the batch size.  
batch_size = 8

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1944: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))


# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions, true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  #label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  #true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 16,973 test sentences...
    DONE.


In [ ]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

In [ ]:
import pickle

with open('/content/drive/MyDrive/rusum/predictions4.pickle', 'wb') as f:
      pickle.dump(flat_predictions, f)

In [ ]:
print(flat_predictions[:10])

[0 0 0 1 0 1 0 1 0 0]


In [ ]:
#write_markup_tsv(predictions, "answer.txt", res_key="quality")
#!rm -f submission_ft.zip
#!zip submission_ft.zip answer.txt